# Show4DSTEM Widget

Interactive 4D-STEM viewer with virtual detector imaging.

In [ ]:
try:
    %load_ext autoreload
    %autoreload 2
    %env ANYWIDGET_HMR=1
except Exception:
    pass  # autoreload unavailable (Colab Python 3.12+)

In [ ]:
import torch
import numpy as np
from quantem.widget import Show4DSTEM

device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")

# Synthetic 4D-STEM: 48x48 scan, 192x192 detector (matches original data shape)
scan_r, scan_c, det_r, det_c = 48, 48, 192, 192
dr = torch.arange(det_r, device=device, dtype=torch.float32)
dc = torch.arange(det_c, device=device, dtype=torch.float32)
rr, cc = torch.meshgrid(dr, dc, indexing="ij")
cr, cc0 = det_r / 2, det_c / 2
dist = ((rr - cr) ** 2 + (cc - cc0) ** 2).sqrt()

# BF disk + Bragg spots + background
base = (dist < 24).float() * 1.0
base += 0.05 * torch.exp(-dist / 60)
for k in range(6):
    angle = k * torch.pi / 3
    sr = cr + 60 * torch.sin(torch.tensor(angle, device=device))
    sc = cc0 + 60 * torch.cos(torch.tensor(angle, device=device))
    base += 0.3 * torch.exp(-((rr - sr) ** 2 + (cc - sc) ** 2) / (2 * 4.0**2))

# Broadcast across scan positions with modulation
si = torch.linspace(0, 1, scan_r, device=device)
sj = torch.linspace(0, 1, scan_c, device=device)
si_g, sj_g = torch.meshgrid(si, sj, indexing="ij")
mod = 1.0 + 0.2 * torch.sin(2 * torch.pi * si_g) * torch.cos(2 * torch.pi * sj_g)
data = base.unsqueeze(0).unsqueeze(0) * mod.unsqueeze(-1).unsqueeze(-1)
if device.type == "mps":
    data = torch.poisson(data.clamp(min=0).cpu() * 100) / 100
else:
    data = torch.poisson(data.clamp(min=0) * 100) / 100
data = data.cpu().numpy().astype(np.float32)

widget = Show4DSTEM(data, pixel_size=2.39, k_pixel_size=0.46)
widget.auto_detect_center()
widget

In [ ]:
# Widget was created with synthetic data + auto-detected center
widget.summary()

## Binned Data

In [4]:
# binned_dataset = dataset.bin(bin_factors=(2, 2), axes=(2, 3))
# widget_binned = Show4DSTEM(binned_dataset)
# widget_binned

## Raster Animation

In [5]:
# widget_anim = Show4DSTEM(dataset)
# widget_anim.raster(step=10, interval_ms=100, loop=True)

## Advanced

Reference for all `Show4DSTEM` parameters and methods.

In [6]:
# # Manual calibration (overrides dataset values)
# widget_config = Show4DSTEM(
#     dataset.array,
#     pixel_size=2.39,      # Å (real-space)
#     k_pixel_size=0.46,    # mrad (k-space)
#     center=(96, 96),
#     bf_radius=24,
#     log_scale=True,
# )
# widget_config

In [ ]:
# ROI methods (chainable)
widget_roi = Show4DSTEM(data, pixel_size=2.39, k_pixel_size=0.46)
widget_roi.auto_detect_center()
widget_roi.roi_annular(inner_radius=15, outer_radius=40)
# widget_roi.roi_point()                    # Single pixel
# widget_roi.roi_circle(radius=20)          # Circle
# widget_roi.roi_square(half_size=15)       # Square (30x30)
# widget_roi.roi_rect(width=20, height=10)  # Rectangle